<a href="https://colab.research.google.com/github/kevinhhl/options-pricing-tools-and-trading-strategies/blob/main/Black_Scholes_Merton_Model_Part1_Screening_YF_for_theoretical_edges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoo_fin

In [ ]:
import math
import time
import pandas as pd
from datetime import date
from scipy.stats import norm
from yahoo_fin import options


# **Overview:**

This script will allow us to obtain options chains provided by Yahoo Finance. We will iterate through the chains, and then apply the Black Scholes Model ("BSM") to calculate the theoretical values and the Greeks. 

**Advantages to BSM:**
* It is widely used
* It's simple to apply

**Disadvantages to BSM:** 
* Was modeled for European-style options; early-exercise is not allowed
* Assumes price to follow a random walk according to a brownian motion with a constant drift. <br>(I have created a separate notebook to explore this topic. [<Link\>](https://github.com/kevinhhl/portfolio-management-tools/blob/main/Monte_Carlo_Simulation_Random_Walk.ipynb))
* Ignores dividends
* Taxes and transaction costs are ignored
* Inputs can be **subjective**, especially for expected volatility

#**Understanding the Model:**

Derivation of the BSM is complex. Through research [1], I'm using my own words to summarize how I understand it from a practical perspective.

The premise of the Black-Scholes Model contains two components. It can be described as taking the difference of: <br>
* (a) the expected value of a stock in the event that it reaches above/below exercise price on the date of expiration for call/put options, respectively, and 
* (b) the expected payout at the exercise price

<br>

These components are expressed as:
<br>\begin{equation}
TV_{call}=se^{rt}N(d_{1})-xN(d_{2})
\end{equation}
<br>\begin{equation}
TV_{put}=xe^{-rt}N(-d_{2})-sN(-d_{1})
\end{equation}

**Where**:
* TV is the theoretical value
* s is the spot price at the current moment
* x is the exercise price of the option
* t is the time till maturity in no. of years
* r is the risk free rate per annum
* σ is the annualized volatility
* N(d) is the CDF of a normal distribution.
<br>
<br>\begin{equation}
d_{1} =\frac{ln(\frac{S}{X})+(r+\frac{\sigma}{2}^2)t}{\sigma\sqrt{t}}
\end{equation}
<br>\begin{equation}
d_{2}=d_{1}-\sigma\sqrt{t}
\end{equation}


*Further contemplations:*
* The probability of price to be in-the-money for call/put options can be pictured as if seeing the data points '*d*' landing in the right/left sides of the CDF (for call/put options, respectively). 
* Probability of the put option to be in-the-money would be N(-d), which represents the left tail of the CDF; speaking of the area from negative-infinity to 0-d. The probability of a call option to be in-the-money would be 1-N(-d); being the right-tail of the CDF.
* Price is not normally distributed. Instead, a lognormal distribution would better describe it. We still want to associate probability of price reaching X with the normal distribution, so we need to make the adjustments to *d*. Also, we will want to risk-adjust *d* according to *r*

<br>

---
*References:*

[1] Natenberg, Sheldon. <i>Chapter 18, Option Volatility and Pricing, Second Edition</i>. McGraw-Hill Edu., 2015.



# **Implementation:**

In [ ]:
class BSM:
  
  def __init__(self, s,t,r,sigma):

    # Fixed variables, will not change as we iterate through the option chain
    self.s = s
    self.t = t
    self.r = r
    self.sigma = sigma

    # variables that will change as we iterate through the option chain
    self.x, self.d1, self.d2 = None, None, None
    self.tv_call, self.delta_call , self.gamma_call, self.vega_call, self.theta_call, self.rho_call = None, None, None, None, None, None
    self.tv_put, self.delta_put, self.gamma_put, self.vega_put, self.theta_put, self.rho_put = None, None, None, None, None, None

  def _recalc(self):
    # _recalc() needs to be called everytime when an input variable is changed
    _a = math.log(self.s/ self.x)
    _b = (self.r+self.sigma**2/2)*self.t
    self.d1 = (_a+_b)/self.sigma*math.sqrt(self.t)
    self.d2 = self.d1 - self.sigma * math.sqrt(self.t)
    
    # Call: TV, delta, gamma, vega, theta, rho
    self.tv_call = self.s * norm.cdf(self.d1) - self.x*math.exp(-self.r*self.t)*norm.cdf(self.d2)
    self.delta_call = norm.cdf(self.d1)
    self.gamma_call = norm.pdf(self.d1)/(self.s*self.sigma*math.sqrt(self.t))
    self.vega_call = 0.01*(self.s*norm.pdf(self.d1)*math.sqrt(self.t))
    self.theta_call = 0.01*(-(self.s*norm.pdf(self.d1)*self.sigma)/(2*math.sqrt(self.t)) - self.r*self.x*math.exp(-self.r*self.t)*norm.cdf(self.d2))
    self.rho_call = 0.01*(self.x*self.t*math.exp(-self.r*self.t)*norm.cdf(self.d2))
    
    # Put: TV, delta, gamma, vega, theta, rho
    self.tv_put = self.x * math.exp(-self.r*self.t)-self.s+self.tv_call
    self.delta_put = -norm.cdf(-self.d1)
    self.gamma_put = norm.pdf(self.d1)/(self.s*self.sigma*math.sqrt(self.t))
    self.vega_put = 0.01*(self.s*norm.pdf(self.d1)*math.sqrt(self.t))
    self.theta_put = 0.01*(-(self.s*norm.pdf(self.d1)*self.sigma)/(2*math.sqrt(self.t)) + self.r*self.x*math.exp(-self.r*self.t)*norm.cdf(-self.d2))
    self.rho_put = 0.01*(-self.x*self.t*math.exp(-self.r*self.t)*norm.cdf(-self.d2))

  def set_x(self, x):
    self.x = x
    self._recalc()


####Validation:


Comparing results with @YuChenAmberLu's implementation [<link\>](https://github.com/YuChenAmberLu/Options-Calculator)

Expecting the results to be identical, if differences (rounded by 5 decimal places) match, then we will be satisfied. 




In [ ]:
test_model = BSM(s=100,t=0.128767,r=0.2,sigma=0.2)
test_model.set_x(100)
assert (test_model.tv_call      -4.112199).round(5)==0
assert (test_model.delta_call   -.520269).round(5)==0
assert (test_model.gamma_call   -.055516).round(5)==0
assert (test_model.vega_call    -.142972).round(5)==0
assert (test_model.theta_call-  -.206861).round(5)==0
assert (test_model.rho_call     -.061698).round(5)==0
assert (test_model.tv_put       -1.569736).round(5)==0
assert (test_model.delta_put-   -.479731).round(5)==0
assert (test_model.gamma_put    -.055516).round(5)==0
assert (test_model.vega_put     -.142972).round(5)==0
assert (test_model.theta_put-   -.011946).round(5)==0
assert (test_model.rho_put-     -.063795).round(5)==0

#Application:
😎 Setting up our model:

In [ ]:
# Manual inputs:
symbol                   = "KO"             # <--  input
riskfree_rate            = .0512            # <--  input
date_today               = date(2023,2,18)  # <--  input
date_expire              = date(2023,2,24)  # <--  input

In [ ]:
# Confirming that the expiration date is valid.
exp_dates = options.get_expiration_dates(symbol)
date_expire_str = date_expire.strftime("%B %d, %Y") 
assert date_expire_str in exp_dates
exp_dates

['February 24, 2023',
 'March 3, 2023',
 'March 10, 2023',
 'March 17, 2023',
 'March 24, 2023',
 'March 31, 2023',
 'April 21, 2023',
 'May 19, 2023',
 'June 16, 2023',
 'July 21, 2023',
 'August 18, 2023',
 'December 15, 2023',
 'January 19, 2024',
 'June 21, 2024',
 'January 17, 2025']

In [ ]:
# Obtaining the recent close price
def get_csv_link(date_from, date_to):
  _date_from = time.mktime(date_from.timetuple())
  _date_to = time.mktime(date_to.timetuple())
  s1 = "https://query1.finance.yahoo.com/v7/finance/download/"
  _s2 = str(int(_date_from))
  _s3 = str(int(_date_to))
  s4 = "?period1="+ _s2 +"&period2="+ _s3 +"&interval=1d&events=history&includeAdjustedClose=true"
  return s1 + symbol + s4

yahoo_csv = pd.read_csv(get_csv_link(date_today, date_today))
assert len(yahoo_csv) == 1
yahoo_csv

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-02-17,59.5,60.23,59.380001,60.119999,60.119999,16589688


In [ ]:
crnt_price = yahoo_csv['Adj Close'][0]
crnt_price

60.119999

In [ ]:
NAME_STRIKE  = "Strike"
NAME_QUOTE   = "Quote"
NAME_TV      = "Th.Value"
NAME_TVDIFF  = "Th.Edge"
NAME_DELTA   = "Δ"
NAME_GAMMA   = "𝚪"
NAME_VEGA    = "V"
NAME_THETA   = "Θ"
NAME_RHO     = "⍴"

FORMAT_COLS = [NAME_QUOTE, NAME_TV, NAME_TVDIFF, NAME_DELTA, NAME_GAMMA, NAME_VEGA, NAME_THETA, NAME_RHO]

def get_df(option_type, date_expire_str, chain, verbose=False):
  l_strike, l_quote, l_tv, l_diff, l_delta, l_gamma, l_vega, l_theta, l_rho, l_index_names = [], [], [], [], [], [], [], [], [], []

  for i in range(len(chain)):
    x = chain["Strike"][i]
    model.set_x(x)    
    
    tv, gamma, theta, vega, theta, rho = None, None, None, None, None, None
    if option_type == "calls":
      tv = model.tv_call
      delta, gamma, vega, theta, rho = model.delta_call, model.gamma_call, model.vega_call, model.theta_call, model.rho_call
    elif option_type == "puts":
      tv = model.tv_put
      delta, gamma, vega, theta, rho = model.delta_put, model.gamma_put, model.vega_put, model.theta_put, model.rho_put
    
    if tv > 0:
      q = chain["Last Price"][i]
      _diff = (q-tv).round(2)
      if verbose:
        print("{}: strike= {}\tquote= {}\tTV= {};\tdiff.= {}".format(option_type, x, q, tv.round(2), _diff ))

      l_index_names.append("{}_{}_{}".format(date_expire_str, x, option_type))
      l_delta.append(delta.round(4))
      l_gamma.append(gamma.round(4))
      l_theta.append(theta.round(4))
      l_vega.append(vega.round(4))
      l_rho.append(rho.round(4))
      l_tv.append(tv.round(2))
      l_diff.append(_diff)
      l_strike.append(x)
      l_quote.append(q)

  return pd.DataFrame({ NAME_STRIKE:l_strike, NAME_QUOTE:l_quote, NAME_TV:l_tv, NAME_TVDIFF:l_diff, \
                       NAME_DELTA:l_delta, NAME_GAMMA:l_gamma, NAME_VEGA:l_vega, NAME_THETA:l_theta, \
                       NAME_RHO:l_rho }, columns=FORMAT_COLS, index=l_index_names)

---
##Option Chain Analysis:

In [ ]:
chain_call = options.get_options_chain(symbol, date_expire_str)["calls"]
chain_put = options.get_options_chain(symbol, date_expire_str)["puts"]

In [ ]:
# get_iv(chain) return float
# Searches for the row that is at-the-money, returns the implied volatility in that row. 
def get_iv(chain):
  iv = None
  for i in range(len(chain)-1):
      c = chain["Strike"]
      if c[i+1] > crnt_price and c[i] < crnt_price:
        iv = chain["Implied Volatility"][i]
        break
  return float(iv.strip("%"))/100

iv = (get_iv(chain_put) + get_iv(chain_call)) / 2
iv

0.14379999999999998

In [ ]:
model = BSM(s     =crnt_price,
            t     =(date_expire-date_today).days/365,
            r     =riskfree_rate,
            sigma =iv )

### Calls:

In [ ]:
df_call = get_df("calls", date_expire_str, chain_call)
print("{} [{}]\n".format(symbol, round(crnt_price,2)))
df_call

KO [60.12]



,Quote,Th.Value,Th.Edge,Δ,𝚪,V,Θ,⍴
"February 24, 2023_35.0_calls",24.50,17.56,6.94,0.6894,0.3258,0.0272,-0.1286,0.0039
"February 24, 2023_50.0_calls",10.95,6.12,4.83,0.5671,0.3629,0.0303,-0.1440,0.0046
"February 24, 2023_54.0_calls",5.55,3.71,1.84,0.5394,0.3663,0.0306,-0.1456,0.0047
"February 24, 2023_55.0_calls",5.20,3.15,2.05,0.5327,0.3669,0.0306,-0.1459,0.0047
"February 24, 2023_56.0_calls",3.87,2.59,1.28,0.5262,0.3673,0.0307,-0.1461,0.0048
"February 24, 2023_57.0_calls",3.01,2.06,0.95,0.5197,0.3677,0.0307,-0.1463,0.0048
"February 24, 2023_57.5_calls",2.51,1.79,0.72,0.5166,0.3678,0.0307,-0.1464,0.0048
"February 24, 2023_58.0_calls",2.13,1.53,0.60,0.5134,0.3679,0.0307,-0.1465,0.0048
"February 24, 2023_59.0_calls",1.20,1.02,0.18,0.5072,0.3680,0.0307,-0.1466,0.0048
"February 24, 2023_60.0_calls",0.52,0.52,0.00,0.5011,0.3681,0.0308,-0.1467,0.0049


### Puts:

In [ ]:
df_put = get_df("puts", date_expire_str, chain_put)
print("{} [{}]\n".format(symbol, round(crnt_price,2)))
df_put

KO [60.12]



,Quote,Th.Value,Th.Edge,Δ,𝚪,V,Θ,⍴
"February 24, 2023_60.0_puts",0.39,0.35,0.04,-0.4989,0.3681,0.0308,-0.1160,-0.0050
"February 24, 2023_61.0_puts",1.01,0.86,0.15,-0.5049,0.3681,0.0307,-0.1155,-0.0051
"February 24, 2023_62.0_puts",1.97,1.38,0.59,-0.5108,0.3680,0.0307,-0.1150,-0.0053
"February 24, 2023_62.5_puts",2.31,1.64,0.67,-0.5138,0.3679,0.0307,-0.1148,-0.0053
"February 24, 2023_63.0_puts",3.55,1.91,1.64,-0.5167,0.3678,0.0307,-0.1145,-0.0054
"February 24, 2023_64.0_puts",3.63,2.46,1.17,-0.5224,0.3675,0.0307,-0.1140,-0.0056
"February 24, 2023_65.0_puts",5.15,3.01,2.14,-0.5280,0.3672,0.0307,-0.1134,-0.0057
"February 24, 2023_68.0_puts",8.43,4.74,3.69,-0.5443,0.3658,0.0306,-0.1115,-0.0062
"February 24, 2023_70.0_puts",10.40,5.95,4.45,-0.5548,0.3646,0.0305,-0.1101,-0.0065
"February 24, 2023_73.0_puts",12.05,7.82,4.23,-0.5699,0.3624,0.0303,-0.1079,-0.0069
